In [24]:
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai
import json
import time
import random

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from flaml.autogen.code_utils import extract_code
from IndustrialChargerLLM.Agent import ChargingAgentAll, ChargingAgentIndividual, reset_params_file, clear_param_backups

In [25]:
benchmark_dataset_path_mgr = "IndustrialChargerLLM/benchmark/EV_combined_mgr.benchmark.json"
benchmark_dataset_path_car = "IndustrialChargerLLM/benchmark/EV_combined_car.benchmark.json"

In [26]:
# global params_filepath
params_filepath = "IndustrialChargerLLM/Model/params/EVCharging.json"
params_filepath_backup = "IndustrialChargerLLM/Model/params/EVCharging_original.json"

In [27]:
with open(params_filepath, 'r') as f:
    
    car_numbers = list(json.loads(f.read())['car_params'].keys())[2:]

In [28]:
car_numbers

['HVC0538',
 'ZNV0672',
 'DXC8753',
 '6TU077',
 '0702YI',
 'P70KBI',
 'KXD3130',
 'QHZ572',
 '6714PP',
 'MS17155']

In [29]:
with open(benchmark_dataset_path_mgr, 'r') as f:
    benchmark_dataset_mgr = json.loads(f.read())

In [30]:
with open(benchmark_dataset_path_car, 'r') as f:
    benchmark_dataset_car = json.loads(f.read())

In [31]:
log_history = {}
autogen.oai.ChatCompletion.start_logging(log_history)

In [32]:
code_path = "IndustrialChargerLLM/Model/EVCharging.py"

In [33]:
with open(code_path) as f:
    code = f.read()

In [34]:
example_qa_all = """
----------
Instruction: Process this request for parking lot manager
Change carbon cost to 30
Answer Code:
```JSON
"lot_params":{"carbon_cost": 30}
```

----------
Instruction: Process this request for car ZNV0672
Update end time to 10 PM
Answer Code:
```JSON
"ZNV0672":{"end_time": 22}
```
"""

In [35]:
example_qa_individual = """
----------
Instruction: 
Change carbon cost to 30
Answer Code:
```JSON
"carbon_cost": 30
```

----------
Instruction: 
Update end time to 10 PM
Answer Code:
```JSON
"end_time": 22
```
"""

In [36]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

In [37]:
config_list_35 = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo"],
    },
)

In [38]:
agentAll = ChargingAgentAll(
    name="Tesla Charging Example",
    source_code=code,
    example_qa=example_qa_all,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
    evaluate = True
)

agentIndividual = ChargingAgentIndividual(
    name="Tesla Charging Example",
    source_code=code,
    example_qa=example_qa_individual,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
    evaluate = True
)

agentAll_35 = ChargingAgentAll(
    name="Tesla Charging Example",
    source_code=code,
    example_qa=example_qa_all,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list_35,
    },
    evaluate = True
)

agentIndividual_35 = ChargingAgentIndividual(
    name="Tesla Charging Example",
    source_code=code,
    example_qa=example_qa_individual,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list_35,
    },
    evaluate = True
)

user_manager = UserProxyAgent(
    "manager", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

user_agents = [UserProxyAgent(
    carNo, max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
) for carNo in car_numbers]

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 68 rows, 480 columns and 1440 nonzeros
Model fingerprint: 0xbfd31c44
Variable types: 0 continuous, 480 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 3e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -486.6400000
Presolve removed 61 rows and 467 columns
Presolve time: 0.00s
Presolved: 7 rows, 13 columns, 19 nonzeros
Found heuristic solution: objective -493.9200000
Variable types: 0 continuous, 13 integer (0 binary)
Found heuristic solution: objective -499.9200000

Root relaxation: objective -5.614400e+02, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

In [39]:
len(benchmark_dataset_car)

207

In [40]:
len(benchmark_dataset_mgr)

364

In [19]:
def parse_json(json_str):
    
    
    try:    
        return json.loads(json_str[7:-4])
    except:
        pass

    try:
        return json.loads('{' + json_str[7:-4] + '}')
    except:
        pass
    
    try:
        return json.loads(json_str )
    except:
        pass

    try:
        return json.loads('{' + json_str + '}')
    except:
        raise TypeError("Invalid String, unable to convert to json") 


In [42]:
benchmark_result_35_all_car = []
sampled_benchmark = random.sample(benchmark_dataset_car, 100)
count_correct = 0
for benchmark_instance in sampled_benchmark:
    for user in user_agents:
    
        log_history = {}
        autogen.oai.ChatCompletion.start_logging(log_history)

        try:
            user.initiate_chat(agentAll_35, message=benchmark_instance['prompt'], silent = True, clear_history=True)
        except:
            benchmark_result_35_all_car.append((0, 'Invalid Response: ' + eval(list(log_history.keys())[-1])[-1]['content']))
            continue

        try:
            predicted_param = parse_json(eval(list(log_history.keys())[-1])[-1]['content'])
        except:
            benchmark_result_35_all_car.append((0, 'Invalid Response: ' + eval(list(log_history.keys())[-1])[-1]['content']))
            continue

        bench_json_str = benchmark_instance['json_str']
        
        truth_param = json.loads('{'+extract_code(bench_json_str)[0][1]+'}')
        result_instance_str = "; bench: " + extract_code(bench_json_str)[0][1] + " pred: " + json.dumps(predicted_param)

        predicted_param_keyCount = len(predicted_param.keys())
        predicted_param_carNo = list(predicted_param.keys())[0]
        predicted_param_value = predicted_param[predicted_param_carNo]
        
        correct_flag = truth_param == predicted_param_value and predicted_param_keyCount == 1 
        correct_flag = correct_flag and predicted_param_carNo == user.name

        result_instance = {'result' : correct_flag, 
                        'index' : benchmark_instance['index'], 
                        'bench' : '{' + extract_code(bench_json_str)[0][1] + '}',
                        'user_name' : user.name,
                        'pred' : json.dumps(predicted_param)}

        count_correct = count_correct + int(correct_flag)
        benchmark_result_35_all_car.append(result_instance)

print("correct: ", str(count_correct))

Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

"HVC0538": {
  "max_charge": 10,
  "end_time": 8
}

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

"ZNV0672":{"max_charge": 10, "end_time": 8}

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

"DXC8753": {
    "max_charge": 10,
    "end_time": 8
}

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------

IndexError: list index out of range

In [47]:
benchmark_result_35_all_car[0]['result']

False

In [55]:
len([x for x in benchmark_result_35_all_car if type(x)!=dict or not x['result']])

325

In [56]:
len(benchmark_result_35_all_car)

509

In [57]:
benchmark_result_35_one_car = []
sampled_benchmark = random.sample(benchmark_dataset_car, 100)
count_correct = 0
for benchmark_instance in sampled_benchmark:
    
    log_history = {}
    autogen.oai.ChatCompletion.start_logging(log_history)

    user = random.choice(user_agents)
    user.initiate_chat(agentIndividual, message=benchmark_instance['prompt'], silent = True, clear_history=True)

    try:
        predicted_param = parse_json(eval(list(log_history.keys())[-1])[-1]['content'])
    except:
        benchmark_result.append((0, 'Invalid Response: ' + eval(list(log_history.keys())[-1])[-1]['content']))

    bench_json_str = benchmark_instance['json_str']
    
    truth_param = json.loads('{'+extract_code(bench_json_str)[0][1]+'}')
    result_instance_str = "; bench: " + extract_code(bench_json_str)[0][1] + " pred: " + json.dumps(predicted_param)
    result_instance = {'result' : truth_param == predicted_param, 
                       'index' : benchmark_instance['index'], 
                       'bench' : '{' + extract_code(bench_json_str)[0][1] + '}',
                       'pred' : json.dumps(predicted_param)}

    count_correct = count_correct + int(truth_param == predicted_param)
    benchmark_result_35_one_car.append(result_instance)

    time.sleep(0.5)
print("correct: ", str(count_correct))


Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
{
  "max_charge": 95,
  "end_time": 14
}
```

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
{
"required_charge": 80,
"max_charge": 70
}
```

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
"required_charge": 55
```

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


---------------------------------------------

In [58]:
count_correct

45

In [59]:
len(benchmark_result_35_one_car)

100

In [60]:
benchmark_result_35_one_car

[{'result': False,
  'index': 175,
  'bench': '{"max_charge": 95,\n"end_charge_time": 14}',
  'pred': '{"max_charge": 95, "end_time": 14}'},
 {'result': True,
  'index': 102,
  'bench': '{"required_charge": 80,\n"max_charge": 70}',
  'pred': '{"required_charge": 80, "max_charge": 70}'},
 {'result': True,
  'index': 5,
  'bench': '{"required_charge": 55}',
  'pred': '{"required_charge": 55}'},
 {'result': True,
  'index': 17,
  'bench': '{"required_charge": 5}',
  'pred': '{"required_charge": 5}'},
 {'result': False,
  'index': 131,
  'bench': '{"required_charge": 65,\n"end_charge_time": 16}',
  'pred': '{"required_charge": 65, "end_time": 16}'},
 {'result': True,
  'index': 8,
  'bench': '{"required_charge": 10}',
  'pred': '{"required_charge": 10}'},
 {'result': True,
  'index': 32,
  'bench': '{"max_charge": 10}',
  'pred': '{"max_charge": 10}'},
 {'result': True,
  'index': 16,
  'bench': '{"required_charge": 25}',
  'pred': '{"required_charge": 25}'},
 {'result': False,
  'index': 

In [32]:
benchmark_result_4_all_car = []
sampled_benchmark = random.sample(benchmark_dataset_car, 100)
count_correct = 0
for benchmark_instance in sampled_benchmark:
    for user in user_agents:
    
        log_history = {}
        autogen.oai.ChatCompletion.start_logging(log_history)

        user.initiate_chat(agentAll, message=benchmark_instance['prompt'], silent = True, clear_history=True)

        try:
            predicted_param = parse_json(eval(list(log_history.keys())[-1])[-1]['content'])
        except:
            benchmark_result_4_all_car.append((0, 'Invalid Response: ' + eval(list(log_history.keys())[-1])[-1]['content']))
            continue

        bench_json_str = benchmark_instance['json_str']
        
        truth_param = json.loads('{'+extract_code(bench_json_str)[0][1]+'}')
        result_instance_str = "; bench: " + extract_code(bench_json_str)[0][1] + " pred: " + json.dumps(predicted_param)

        predicted_param_keyCount = len(predicted_param.keys())
        predicted_param_carNo = list(predicted_param.keys())[0]
        predicted_param_value = predicted_param[predicted_param_carNo]
        
        correct_flag = truth_param == predicted_param_value and predicted_param_keyCount == 1 
        correct_flag = correct_flag and predicted_param_carNo == user.name

        result_instance = {'result' : correct_flag, 
                        'index' : benchmark_instance['index'], 
                        'bench' : '{' + extract_code(bench_json_str)[0][1] + '}',
                        'user_name' : user.name,
                        'pred' : json.dumps(predicted_param)}

        count_correct = count_correct + int(truth_param == predicted_param)
        benchmark_result_4_all_car.append(result_instance)



Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
"HVC0538":{
    "required_charge": 20,
    "end_time": 10
}
```

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
"ZNV0672": {"required_charge": 20, "end_time": 10}
```

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
"DXC8753":{"required_charge": 20, "end_time": 10}
```

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer

KeyboardInterrupt: 

In [78]:
[benchmark_dataset[result['index']] for result in benchmark_result if not result['result']]

[{'json_str': '```JSON\n"carbon_cost_weight": 0.0,\n"battery_capacity": 40\n```',
  'prompt': 'I want to not consider carbon emissions while charging and change the battery capacity to 90 KWH.',
  'index': 297},
 {'json_str': '```JSON\n"end_charge": 0.35,\n"end_charge_time": 0\n```',
  'prompt': 'Start charging the car immediately and stop when it reaches 35% battery level.',
  'index': 704},
 {'json_str': '```JSON\n"end_charge": 0.7,\n"max_power": 17.8\n```',
  'prompt': 'Change the value of "end_charge" to 0.7.',
  'index': 572},
 {'json_str': '```JSON\n"carbon_cost_weight": 0.0,\n"end_charge": 0.95\n```',
  'prompt': 'Change the value of the "end_charge" parameter to 0.95.',
  'index': 174},
 {'json_str': '```JSON\n"max_power": 10.7,\n"battery_capacity": 15\n```',
  'prompt': 'The change is to update the parameters "max_power" to 11.7 and "battery_capacity" to 70.',
  'index': 751},
 {'json_str': '```JSON\n"carbon_cost_weight": 0.15,\n"end_charge": 0.5\n```',
  'prompt': 'Update the

In [85]:
[result['pred'] + ";" + benchmark_dataset[result['index']]['prompt'] for result in benchmark_result if not result['result']]

['{"carbon_cost_weight": 0, "battery_capacity": 90};I want to not consider carbon emissions while charging and change the battery capacity to 90 KWH.',
 '{"start_charge": 0.0, "end_charge": 0.35};Start charging the car immediately and stop when it reaches 35% battery level.',
 '{"end_charge": 0.7};Change the value of "end_charge" to 0.7.',
 '{"end_charge": 0.95};Change the value of the "end_charge" parameter to 0.95.',
 '{"max_power": 11.7, "battery_capacity": 70};The change is to update the parameters "max_power" to 11.7 and "battery_capacity" to 70.',
 '{"carbon_cost_weight": 0.15, "end_charge": 1.0};Update the weight of carbon cost to 15% and charge my car to full battery.',
 '{"carbon_cost_weight": 0.85, "max_power": 11.3};Increase the importance of carbon cost in total cost to 85% and reduce the maximum charge power to 11.3 KW.',
 '{"max_power": 18.8};Increase the max charge power to 18.8 KW while leaving the battery capacity unchanged at 70 KWH.',
 '{"max_power": 17.3, "end_charg

In [100]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo-16k"],
    },
)

In [104]:
agent_gpt3 = ChargingAgent(
    name="Tesla Charging Example",
    source_code=code,
    example_qa=example_qa,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
    evaluate=True
)

{'0': 0.3, '1': 0.3, '2': 0.3, '3': 0.3, '4': 0.3, '5': 0.3, '6': 0.3, '7': 0.35, '8': 0.35, '9': 0.35, '10': 0.35, '11': 0.3, '12': 0.3, '13': 0.3, '14': 0.3, '15': 0.3, '16': 0.3, '17': 0.3, '18': 0.3, '19': 0.35, '20': 0.35, '21': 0.35, '22': 0.3, '23': 0.3}
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 48 columns and 96 nonzeros
Model fingerprint: 0x7c396a84
Variable types: 0 continuous, 48 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [8e+00, 8e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [2e+01, 4e+01]
Presolve removed 2 rows and 48 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 327.6 

Optimal solution found (toleran

In [102]:
log_history = {}
autogen.oai.ChatCompletion.start_logging(log_history)

In [105]:
benchmark_result_gpt3 = []
count_correct_gpt3 = 0
for benchmark_instance in sampled_benchmark:
    
    log_history = {}
    autogen.oai.ChatCompletion.start_logging(log_history)

    user.initiate_chat(agent_gpt3, message=benchmark_instance['prompt'], silent = True, clear_history=True)

    try:
        predicted_param = parse_json(eval(list(log_history.keys())[-1])[-1]['content'])
    except:
        benchmark_result.append((0, 'Invalid Response: ' + eval(list(log_history.keys())[-1])[-1]['content']))

    bench_json_str = benchmark_instance['json_str']
    
    truth_param = json.loads('{'+extract_code(bench_json_str)[0][1]+'}')
    result_instance_str = "; bench: " + extract_code(bench_json_str)[0][1] + " pred: " + json.dumps(predicted_param)
    result_instance = {'result' : truth_param == predicted_param, 
                       'index' : benchmark_instance['index'], 
                       'bench' : '{' + extract_code(bench_json_str)[0][1] + '}',
                       'pred' : json.dumps(predicted_param)}

    count_correct_gpt3 = count_correct_gpt3 + int(truth_param == predicted_param)
    benchmark_result_gpt3.append(result_instance)

    time.sleep(0.5)


Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

"end_charge": 0.55,
"battery_capacity": 25

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

{
    "end_charge": 0.65,
    "max_power": 14.3
}

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
{
    "carbon_cost_weight": 1.15,
    "max_energy_cost": 12
}
```

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


---------------------

In [107]:
count_correct_gpt3

24

In [90]:
benchmark_result_gpt3

[{'result': True,
  'index': 641,
  'bench': '{"end_charge": 0.55,\n"battery_capacity": 25}',
  'pred': '{"end_charge": 0.55, "battery_capacity": 25}'},
 {'result': True,
  'index': 590,
  'bench': '{"end_charge": 0.65,\n"max_power": 14.3}',
  'pred': '{"end_charge": 0.65, "max_power": 14.3}'},
 {'result': True,
  'index': 155,
  'bench': '{"carbon_cost_weight": 1.15,\n"max_energy_cost": 12}',
  'pred': '{"carbon_cost_weight": 1.15, "max_energy_cost": 12}'},
 {'result': True,
  'index': 87,
  'bench': '{"battery_capacity": 55}',
  'pred': '{"battery_capacity": 55}'},
 {'result': True,
  'index': 543,
  'bench': '{"max_energy_cost": 52,\n"end_charge_time": 3}',
  'pred': '{"end_charge_time": 3, "max_energy_cost": 52}'},
 {'result': True,
  'index': 659,
  'bench': '{"end_charge": 0.7,\n"end_charge_time": 20}',
  'pred': '{"end_charge": 0.7, "end_charge_time": 20}'},
 {'result': True,
  'index': 122,
  'bench': '{"carbon_cost_weight": 0.45,\n"max_energy_cost": 97}',
  'pred': '{"carbon_c

In [108]:
[result['pred'] + ";" + benchmark_dataset[result['index']]['prompt'] for result in benchmark_result_gpt3 if not result['result']][:10]

['{"carbon_cost_weight": 0.35, "battery_capacity": 90};Charge the electric car to 55% and set the battery capacity to 25 KWH.',
 '{"carbon_cost_weight": 0.35, "battery_capacity": 90};Charge the car until it reaches 65% battery level and increase the maximum charge power to 14.3 KW.',
 '{"carbon_cost_weight": 1.15, "max_energy_cost": 12};Battery capacity is updated to 55 kWh.',
 '{"carbon_cost_weight": 1.15, "max_energy_cost": 12};Charge the car until 3 AM with a maximum energy cost of 52 USD.',
 '{"carbon_cost_weight": 1.15, "max_energy_cost": 12};Charge the car till 70% and complete it by 8 PM.',
 '{"carbon_cost_weight": 1.15, "max_energy_cost": 12};Increase the weight of carbon cost in total cost to 0.45 and raise the maximum electricity cost allowed per charge cycle to $97.',
 '{"carbon_cost_weight": 1.15, "max_energy_cost": 12};Charge the car until 3 PM.',
 '{"carbon_cost_weight": 0.7, "end_charge": 0.95};Increase the weightage of carbon cost to 0.35 and upgrade the car\'s battery 

In [109]:
with open('benchmark/results/gpt4_20231203.json', 'w') as f:
    f.write(json.dumps(benchmark_result))

In [110]:
with open('benchmark/results/gpt35_20231203.json', 'w') as f:
    f.write(json.dumps(benchmark_result_gpt3))